In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T
import torchvision.models as models
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler  # or StandardScaler

In [ ]:
# -----------------------
# 1. Custom Dataset
# -----------------------
class PovertyDataset(Dataset):
    """
    Example dataset structure:
    - images_dir: Path to folder containing images. Each image has 4 channels (RGB + vegetation).
    - targets: A list/array of the poverty values (regression targets), 
               aligned with the images in images_dir by index or filename.

    For demonstration, we assume:
      - 'images_filenames' is a list of image file paths
      - 'targets' is a list/array of target poverty values
    """

    def __init__(self, images_filenames, targets, transform=None):
        self.images_filenames = images_filenames
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.images_filenames)

    def __getitem__(self, idx):
        # Load the image (4 channel). 
        # If your files are not standard image formats, you may need special loaders (e.g. tif)
        img_path = self.images_filenames[idx]
        with Image.open(img_path) as img:
            # Ensure the image is in "RGBA" or 4-channel mode if needed
            # If you have a separate vegetation channel, you might combine them manually
            # or read a 4-band GeoTIFF.
            # For demonstration, we assume a 4-channel image is read directly.
            img = img.convert("RGBA")  # or keep it as 4-ch if your PIL is reading a 4-band image

            # Convert to numpy array (H, W, C)
            img_np = np.array(img, dtype=np.float32)
            
            # If the vegetation channel is separate, you'll have to read that separately 
            # and stack them. Or if it's already included as the 4th band, skip this step.

        # target
        target = self.targets[idx]

        # Apply transform if available (including scaling, etc.)
        if self.transform:
            img_np = self.transform(img_np)  # e.g. transforms on numpy or tensor

        return img_np, np.float32(target)


# -----------------------
# 2. Define Transforms
# -----------------------
class ToTensor:
    """Convert a numpy array (H,W,C) to a PyTorch tensor of shape (C,H,W)."""
    def __call__(self, sample):
        # sample shape is (H, W, C)
        sample_tensor = torch.from_numpy(sample).permute(2, 0, 1)  # (C, H, W)
        return sample_tensor

# Example of a scaling transform using MinMax
class MinMaxScale:
    """
    This transform scales each channel to [0,1] (or a certain range).
    Alternatively, you could do channel-wise normalization.
    """
    def __init__(self, min_val=None, max_val=None):
        # If you know min/max across dataset, set them here
        # or compute them in an offline step
        self.min_val = min_val if min_val is not None else 0.0
        self.max_val = max_val if max_val is not None else 255.0

    def __call__(self, sample_tensor):
        return (sample_tensor - self.min_val) / (self.max_val - self.min_val + 1e-8)

# Compose the transforms
transform = T.Compose([
    ToTensor(),           # convert HWC -> CHW
    MinMaxScale(0, 255),  # scale pixel values
])


# -----------------------
# 3. Prepare Data & Splits
# -----------------------

# Suppose you have these lists from your data pipeline
# images_filenames = ["path/to/image1.png", "path/to/image2.png", ...]
# targets = [poverty_value1, poverty_value2, ...]

images_filenames = [...]  # fill in
targets = [...]           # fill in

# Using sklearn to split into train/test
train_files, test_files, train_targets, test_targets = train_test_split(
    images_filenames, targets, test_size=0.2, random_state=42
)

# Create dataset objects
train_dataset = PovertyDataset(train_files, train_targets, transform=transform)
test_dataset  = PovertyDataset(test_files,  test_targets,  transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=8, shuffle=False)


# -----------------------
# 4. Define/Modify Pretrained Model
# -----------------------
# We’ll use ResNet18 here, but you can pick other architectures.

model = models.resnet18(pretrained=True)

# 4.1 Modify the first conv layer to accept 4 channels
# The original resnet18 first layer is:
#   model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
# We change it to have in_channels=4
old_weights = model.conv1.weight.data  # shape: (64, 3, 7, 7)

# Create a new conv layer
new_conv = nn.Conv2d(
    in_channels=4,        # 4 channels now
    out_channels=64,
    kernel_size=7,
    stride=2,
    padding=3,
    bias=False
)

# Copy the original RGB weights
# old_weights[:, :3, :, :] is shape: (64, 3, 7, 7)
new_conv.weight.data[:, :3, :, :] = old_weights
# Initialize the weights of the 4th channel to something small (e.g. zeros or random)
nn.init.xavier_normal_(new_conv.weight.data[:, 3:, :, :])

model.conv1 = new_conv

# 4.2 Replace the final classification layer (fc) with a regression head
# Original fully connected layer: model.fc = nn.Linear(512, 1000)
# For regression, let's output a single value
num_feats = model.fc.in_features
model.fc = nn.Linear(num_feats, 1)  # single regression output


# -----------------------
# 5. Set up Training
# -----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training parameters
num_epochs = 5  # for example
train_losses = []
test_losses = []

# -----------------------
# 6. Training Loop
# -----------------------
for epoch in range(num_epochs):
    # --- TRAIN ---
    model.train()
    running_train_loss = 0.0
    for images, targets in train_loader:
        images = images.to(device)
        targets = targets.to(device).view(-1, 1)  # shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item() * images.size(0)

    epoch_train_loss = running_train_loss / len(train_loader.dataset)
    train_losses.append(epoch_train_loss)

    # --- EVAL ---
    model.eval()
    running_test_loss = 0.0
    with torch.no_grad():
        for images, targets in test_loader:
            images = images.to(device)
            targets = targets.to(device).view(-1, 1)

            outputs = model(images)
            loss = criterion(outputs, targets)
            running_test_loss += loss.item() * images.size(0)

    epoch_test_loss = running_test_loss / len(test_loader.dataset)
    test_losses.append(epoch_test_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Train Loss: {epoch_train_loss:.4f} | Test Loss: {epoch_test_loss:.4f}")


# -----------------------
# 7. Plotting
# -----------------------
plt.figure(figsize=(8, 5))
plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs+1), test_losses, label='Test Loss')
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Train vs Test Loss")
plt.legend()
plt.show()